Information for running: NoteBook 
If you already have tables of the same name, these tables will be replaced or dropped

Instructions:  
    This notebook uses MongoDB on localhost port 27017
    
    ** change connect string if port is different
    
    
  This is not the finished Notebook, this is only a starting point to see what is in the dataset .
    We can merge the data before adding to the collection, make a collection of only cities and states
    and other information in the collection
    MonogoDB is a document database (noSQL) non relational so there is no need for one from one relationship.
    We could link the data by state and pull attributes from each collection from that state to display in a dataframe
    Then load then present that data in the form of graphs to the respective HTML pages 
    
    Current data loads from the following CSVs:
    Resources/accidentalDeath.csv
    Resources/gunOwnershipByState2022.csv
    
    
    Still need : Suicide data and any other data points required. 

In [1]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas -- need to install this if you do not have it

#import requests
import json
import pandas as pd

#import time
#import os
#import psycopg2
import pymongo
#from sqlalchemy import create_engine
#from dbconnection import connect_string


#import sqlalchemy
#from sqlalchemy.ext.automap import automap_base
#from sqlalchemy.orm import Session
#from sqlalchemy import create_engine,inspect, func

In [2]:
import pandas as pd
from pymongo import MongoClient  # remember to move this to the top
import json


In [3]:
#create connection strong to mongoDB database
client = pymongo.MongoClient ("mongodb://localhost:27017")


In [4]:
# read accidental death information from CSV
df = pd.read_csv("Resources/accidentalDeath.csv")

In [5]:
df.head()

,Incident ID,Incident Date,State,City Or County,Address,# Killed,# Injured,Operations
0,2424304,25-Sep-22,Virginia,Suffolk,200 block of Pine St,1,0,NaN
1,2424013,25-Sep-22,Ohio,Saint Clairsville,139 N Market St,1,0,NaN
2,2424073,24-Sep-22,Wisconsin,Milwaukee,6900 block of N 43rd St,1,0,NaN
3,2423209,24-Sep-22,Michigan,Sterling Heights,43000 block of Frontenac Ave,1,0,NaN
4,2424598,23-Sep-22,California,South Lake Tahoe,700 block of James Ave,1,0,NaN


In [6]:
df.tail()

,Incident ID,Incident Date,State,City Or County,Address,# Killed,# Injured,Operations
1995,1215754,22-Sep-18,Colorado,Grand Junction,1100 block of Main St,1,0,NaN
1996,1216441,22-Sep-18,Michigan,Linden,16000 block of Seymour Rd,1,0,NaN
1997,1213916,19-Sep-18,Illinois,Rockford,2300 block of Clover Ave,1,0,NaN
1998,1213277,18-Sep-18,Ohio,Bellevue,200 block of Kilbourne St,1,0,NaN
1999,1211994,17-Sep-18,Louisiana,Livingston,18000 block of Murphy Estate Ln,1,0,NaN


In [7]:
df.shape   # # of rows and columns

(2000, 8)

In [8]:
# convert data to key value pairs to import into mongoDB
data = df.to_dict(orient="records")  # convert to dicitionary and pass parameter 

In [9]:
data

[{'Incident ID': 2424304,
  'Incident Date': '25-Sep-22',
  'State': 'Virginia',
  'City Or County': 'Suffolk',
  'Address': '200 block of Pine St',
  '# Killed': 1,
  '# Injured': 0,
  'Operations': nan},
 {'Incident ID': 2424013,
  'Incident Date': '25-Sep-22',
  'State': 'Ohio',
  'City Or County': 'Saint Clairsville',
  'Address': '139 N Market St',
  '# Killed': 1,
  '# Injured': 0,
  'Operations': nan},
 {'Incident ID': 2424073,
  'Incident Date': '24-Sep-22',
  'State': 'Wisconsin',
  'City Or County': 'Milwaukee',
  'Address': '6900 block of N 43rd St',
  '# Killed': 1,
  '# Injured': 0,
  'Operations': nan},
 {'Incident ID': 2423209,
  'Incident Date': '24-Sep-22',
  'State': 'Michigan',
  'City Or County': 'Sterling Heights',
  'Address': '43000 block of Frontenac Ave',
  '# Killed': 1,
  '# Injured': 0,
  'Operations': nan},
 {'Incident ID': 2424598,
  'Incident Date': '23-Sep-22',
  'State': 'California',
  'City Or County': 'South Lake Tahoe',
  'Address': '700 block of Ja

In [10]:
db=client["crimestats"]  # stage DB for mongoDB

In [11]:
print (db)   # verify DB create

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'crimestats')


In [12]:
# check if a collection exists
col_exists = 'accDeathByState' in db.list_collection_names()

print ("'Does accDeathByState' Exists ", col_exists) # will print True or False

# use the database_name.some_collection.drop() method call if True
if col_exists == True:
# get the collection object if it exists
    print ("'accDeathByState' Dropping database ") # will print True or False
    col = db['accDeathByStateaccDeathByState']

# drop the collection
    col.drop()


'Does accDeathByState' Exists  True
'accDeathByState' Dropping database 


In [13]:
col_exists2 = 'gunsbystate' in db.list_collection_names()
if col_exists2 == True:
# get the collection object if it exists
    print ("'gunsbystate' Dropping database ") # will print True or False
    col = db['gunsbystate']

# drop the collection
    col.drop()


'gunsbystate' Dropping database 


In [14]:
db.accDeathByState.insert_many(data)
# accDeathByState is the name of the collection
#db is the variable for the database name 

In [15]:
# create second dataframe to read and hold information
df2 = pd.read_csv("Resources/gunOwnershipByState2022.csv")

In [16]:
df2.head()

,State,gunOwnership,totalGuns
0,Montana,0.663,22133
1,Wyoming,0.662,132806
2,Alaska,0.645,15824
3,Idaho,0.601,49566
4,West Virginia,0.585,35264


In [17]:
df2.count()  # count of records in dataframe

State           50
gunOwnership    50
totalGuns       50
dtype: int64

In [18]:
# helpful to review to understand orient https://www.geeksforgeeks.org/python-pandas-dataframe-to_dict/ 
#and https://pandas.pydata.org/pandas-docs/dev/reference/api/pandas.DataFrame.to_json.html

data2 = df2.to_dict(orient="records")  # convert dataframe in to record dictionary format

In [19]:
db.gunsbystate.insert_many(data2)   # create  collection "gunByDate"  and load data 

In [20]:
# Verify that the data can be pulled from document database (noSQL) Database is MonogoDB
accDbCol = db["accDeathByState"]   # Define variable to hold collection

x = accDbCol.find()   # define variable to hold list to loop through information to find data in collection

for data in x:   # use for loop to loop through records in database
    print(data['State'], data['# Killed'], data['Incident Date'])  # pull the following information 

Virginia 1 25-Sep-22
Ohio 1 25-Sep-22
Wisconsin 1 24-Sep-22
Michigan 1 24-Sep-22
California 1 23-Sep-22
Colorado 1 23-Sep-22
Florida 1 21-Sep-22
South Carolina 1 21-Sep-22
Virginia 1 17-Sep-22
North Carolina 1 16-Sep-22
Texas 1 16-Sep-22
New Jersey 1 16-Sep-22
New Jersey 1 16-Sep-22
Pennsylvania 1 14-Sep-22
Ohio 1 14-Sep-22
Michigan 1 13-Sep-22
Virginia 1 12-Sep-22
Illinois 1 10-Sep-22
Tennessee 1 10-Sep-22
Maine 1 9-Sep-22
Texas 1 9-Sep-22
Nevada 1 9-Sep-22
Texas 1 5-Sep-22
Indiana 1 4-Sep-22
Oklahoma 1 4-Sep-22
New York 1 2-Sep-22
Indiana 1 1-Sep-22
Missouri 1 31-Aug-22
Minnesota 1 29-Aug-22
Alabama 1 29-Aug-22
New York 1 28-Aug-22
Alaska 1 28-Aug-22
Hawaii 1 28-Aug-22
Florida 1 27-Aug-22
West Virginia 1 26-Aug-22
Arizona 1 24-Aug-22
Florida 1 24-Aug-22
Tennessee 1 22-Aug-22
Michigan 1 22-Aug-22
Michigan 1 22-Aug-22
Georgia 1 22-Aug-22
Ohio 1 21-Aug-22
California 1 21-Aug-22
Florida 1 20-Aug-22
California 1 19-Aug-22
Mississippi 1 19-Aug-22
Maryland 1 18-Aug-22
Pennsylvania 1 17-Aug-

Virginia 1 1-Feb-20
Tennessee 1 1-Feb-20
Texas 1 31-Jan-20
Pennsylvania 1 30-Jan-20
Alabama 1 30-Jan-20
North Carolina 1 28-Jan-20
Washington 1 28-Jan-20
Ohio 1 26-Jan-20
Ohio 1 26-Jan-20
Louisiana 1 25-Jan-20
Ohio 1 25-Jan-20
Arizona 1 23-Jan-20
Louisiana 1 22-Jan-20
Georgia 1 22-Jan-20
Texas 1 22-Jan-20
Texas 1 22-Jan-20
Washington 1 22-Jan-20
Wisconsin 1 20-Jan-20
North Carolina 1 20-Jan-20
Tennessee 1 19-Jan-20
Indiana 1 19-Jan-20
Texas 1 17-Jan-20
Georgia 1 17-Jan-20
California 1 17-Jan-20
Massachusetts 1 15-Jan-20
Texas 1 14-Jan-20
Florida 1 12-Jan-20
California 1 11-Jan-20
North Carolina 1 11-Jan-20
Kentucky 1 8-Jan-20
Maryland 1 6-Jan-20
Missouri 1 5-Jan-20
Tennessee 1 4-Jan-20
Maryland 1 3-Jan-20
Virginia 1 1-Jan-20
South Carolina 2 1-Jan-20
Tennessee 1 1-Jan-20
Virginia 1 1-Jan-20
Texas 1 1-Jan-20
Maryland 1 1-Jan-20
Ohio 1 1-Jan-20
Tennessee 1 31-Dec-19
Oklahoma 1 29-Dec-19
Missouri 1 28-Dec-19
Texas 1 25-Dec-19
Alabama 1 24-Dec-19
Texas 1 22-Dec-19
Texas 1 21-Dec-19
Califor

Ohio 1 2-Dec-20
Pennsylvania 1 1-Dec-20
Missouri 1 30-Nov-20
Nevada 1 29-Nov-20
Florida 1 29-Nov-20
Oklahoma 1 28-Nov-20
Montana 1 28-Nov-20
Wisconsin 1 28-Nov-20
Maryland 1 28-Nov-20
Texas 1 26-Nov-20
Michigan 2 26-Nov-20
Tennessee 1 25-Nov-20
Georgia 1 25-Nov-20
Minnesota 1 25-Nov-20
Texas 1 24-Nov-20
Wyoming 1 23-Nov-20
Arizona 1 23-Nov-20
Arkansas 1 23-Nov-20
Wisconsin 1 22-Nov-20
Mississippi 1 21-Nov-20
Indiana 1 21-Nov-20
Florida 1 21-Nov-20
Pennsylvania 1 20-Nov-20
Michigan 1 20-Nov-20
Mississippi 1 20-Nov-20
New York 1 19-Nov-20
Nevada 1 18-Nov-20
Texas 1 18-Nov-20
Oklahoma 1 17-Nov-20
New York 1 16-Nov-20
Florida 1 15-Nov-20
South Carolina 1 14-Nov-20
Michigan 1 14-Nov-20
Texas 1 14-Nov-20
Minnesota 1 13-Nov-20
Texas 1 13-Nov-20
Nebraska 1 11-Nov-20
Mississippi 1 11-Nov-20
North Carolina 1 11-Nov-20
Virginia 1 10-Nov-20
Missouri 1 9-Nov-20
Colorado 1 9-Nov-20
California 1 8-Nov-20
Texas 1 8-Nov-20
New York 1 7-Nov-20
Utah 1 6-Nov-20
Georgia 1 6-Nov-20
Louisiana 2 6-Nov-20
Kans

North Carolina 1 1-Oct-21
Florida 1 30-Sep-21
Mississippi 1 29-Sep-21
Wisconsin 1 28-Sep-21
Indiana 1 28-Sep-21
New York 1 28-Sep-21
Kansas 1 27-Sep-21
Texas 1 26-Sep-21
Georgia 1 25-Sep-21
Texas 1 22-Sep-21
Arkansas 1 19-Sep-21
Wisconsin 1 18-Sep-21
Colorado 1 17-Sep-21
Kansas 1 17-Sep-21
Texas 1 16-Sep-21
Illinois 1 15-Sep-21
New York 1 15-Sep-21
Georgia 1 13-Sep-21
Florida 1 13-Sep-21
North Carolina 1 12-Sep-21
Arkansas 1 12-Sep-21
Alaska 1 11-Sep-21
Tennessee 1 11-Sep-21
California 1 11-Sep-21
Georgia 1 10-Sep-21
Mississippi 1 9-Sep-21
Illinois 1 9-Sep-21
Missouri 1 8-Sep-21
Texas 1 8-Sep-21
Texas 1 7-Sep-21
New York 1 4-Sep-21
Texas 1 4-Sep-21
North Carolina 1 3-Sep-21
Texas 1 3-Sep-21
Florida 1 3-Sep-21
Arizona 1 3-Sep-21
Michigan 1 1-Sep-21
Illinois 1 1-Sep-21
Indiana 1 31-Aug-21
Florida 1 29-Aug-21
Michigan 1 29-Aug-21
New Jersey 1 29-Aug-21
Oklahoma 1 29-Aug-21
Oklahoma 1 28-Aug-21
Tennessee 1 27-Aug-21
Tennessee 1 27-Aug-21
Mississippi 2 26-Aug-21
Florida 1 26-Aug-21
Florida 

Oregon 1 24-Nov-18
South Carolina 1 24-Nov-18
Alabama 1 24-Nov-18
Georgia 1 22-Nov-18
Florida 1 21-Nov-18
Montana 1 21-Nov-18
North Carolina 1 20-Nov-18
Colorado 1 19-Nov-18
North Carolina 1 18-Nov-18
Texas 1 18-Nov-18
Arkansas 1 17-Nov-18
Texas 1 17-Nov-18
Michigan 1 16-Nov-18
Georgia 1 15-Nov-18
Michigan 1 15-Nov-18
New York 1 15-Nov-18
Arizona 1 14-Nov-18
Alabama 1 14-Nov-18
Louisiana 1 13-Nov-18
Illinois 1 12-Nov-18
Idaho 1 11-Nov-18
Arkansas 1 11-Nov-18
Missouri 1 11-Nov-18
Nevada 1 10-Nov-18
Missouri 1 10-Nov-18
Nebraska 1 10-Nov-18
Louisiana 1 10-Nov-18
Missouri 1 10-Nov-18
Minnesota 1 10-Nov-18
Tennessee 1 9-Nov-18
New York 1 9-Nov-18
Arkansas 1 8-Nov-18
Missouri 1 6-Nov-18
Colorado 1 5-Nov-18
Washington 1 4-Nov-18
Alabama 1 4-Nov-18
Florida 1 4-Nov-18
Oregon 1 4-Nov-18
Louisiana 1 2-Nov-18
Montana 1 2-Nov-18
Colorado 1 1-Nov-18
Wisconsin 1 31-Oct-18
New York 1 30-Oct-18
Alabama 1 29-Oct-18
West Virginia 1 25-Oct-18
Georgia 1 22-Oct-18
Colorado 1 21-Oct-18
Montana 1 21-Oct-18
T

Pennsylvania 1 1-Aug-20
Illinois 1 31-Jul-20
Maryland 1 31-Jul-20
Illinois 1 31-Jul-20
Maryland 1 31-Jul-20
Indiana 1 29-Jul-20
Tennessee 1 28-Jul-20
Georgia 1 28-Jul-20
Hawaii 1 28-Jul-20
Arizona 1 27-Jul-20
Tennessee 1 26-Jul-20
Ohio 1 26-Jul-20
California 1 26-Jul-20
Florida 1 25-Jul-20
South Carolina 1 25-Jul-20
New York 1 25-Jul-20
Wisconsin 1 24-Jul-20
Wisconsin 1 24-Jul-20
Nebraska 1 23-Jul-20
Tennessee 1 22-Jul-20
Michigan 1 22-Jul-20
South Carolina 1 22-Jul-20
Utah 1 22-Jul-20
Colorado 1 22-Jul-20
Wisconsin 1 21-Jul-20
Texas 1 21-Jul-20
Illinois 1 21-Jul-20
Texas 1 20-Jul-20
Nevada 1 19-Jul-20
Texas 1 19-Jul-20
Georgia 1 19-Jul-20
Florida 1 19-Jul-20
South Carolina 1 18-Jul-20
North Carolina 1 18-Jul-20
Louisiana 1 18-Jul-20
Colorado 1 16-Jul-20
Texas 1 16-Jul-20
Florida 1 15-Jul-20
Alabama 1 15-Jul-20
Missouri 1 15-Jul-20
Georgia 1 14-Jul-20
Virginia 1 14-Jul-20
Colorado 1 13-Jul-20
North Carolina 1 13-Jul-20
Washington 1 13-Jul-20
Oklahoma 1 13-Jul-20
Illinois 1 12-Jul-20
Ke

Louisiana 1 27-Aug-19
Tennessee 1 27-Aug-19
Pennsylvania 1 26-Aug-19
Texas 1 26-Aug-19
Pennsylvania 1 26-Aug-19
Alabama 1 25-Aug-19
Florida 1 24-Aug-19
Alabama 1 23-Aug-19
Idaho 1 23-Aug-19
Florida 1 23-Aug-19
Illinois 1 21-Aug-19
Pennsylvania 1 20-Aug-19
Minnesota 1 19-Aug-19
Florida 1 18-Aug-19
California 1 17-Aug-19
Oklahoma 1 17-Aug-19
Kansas 1 17-Aug-19
Ohio 1 17-Aug-19
Texas 1 16-Aug-19
Alabama 1 16-Aug-19
West Virginia 1 14-Aug-19
Arizona 1 14-Aug-19
Utah 1 14-Aug-19
Texas 1 14-Aug-19
Nevada 1 12-Aug-19
Michigan 1 11-Aug-19
Ohio 1 11-Aug-19
Ohio 1 10-Aug-19
Kansas 1 8-Aug-19
Illinois 1 8-Aug-19
New York 1 6-Aug-19
Alabama 1 5-Aug-19
Louisiana 1 4-Aug-19
Virginia 1 2-Aug-19
Texas 1 1-Aug-19
Montana 1 1-Aug-19
Arkansas 1 1-Aug-19
Arizona 1 30-Jul-19
Arizona 1 30-Jul-19
Pennsylvania 1 29-Jul-19
Illinois 1 28-Jul-19
Alaska 1 27-Jul-19
Illinois 1 26-Jul-19
Texas 1 26-Jul-19
New Jersey 1 25-Jul-19
Texas 1 24-Jul-19
Texas 1 24-Jul-19
Ohio 1 24-Jul-19
Georgia 1 22-Jul-19
Louisiana 1 21-

In [21]:
df3 = pd.read_csv("Resources/massShootings.csv")

In [22]:
df3.head()

,Incident ID,Incident Date,State,City Or County,Address,# Killed,# Injured,Operations
0,2425199,27-Sep-22,Pennsylvania,Philadelphia,6498 Ridge Ave,1,4,NaN
1,2425393,27-Sep-22,New York,Corona (Queens),88th St and Atlantic Ave,0,4,NaN
2,2423747,25-Sep-22,Pennsylvania,Harrisburg,2300 block of Woodlawn St,0,4,NaN
3,2423338,25-Sep-22,Indiana,Hammond,6217 Kennedy Ave,1,3,NaN
4,2423538,24-Sep-22,Ohio,Columbus,E 5th Ave and Osborn Ave,1,3,NaN


In [23]:
col_exists3 = 'massShootings' in db.list_collection_names()
if col_exists3 == True:
# get the collection object if it exists
    print ("'massShootings' Dropping database ") # will print True or False
    col = db['massShootings']

# drop the collection
    col.drop()


'massShootings' Dropping database 


In [24]:
data3 = df3.to_dict(orient="records")

In [25]:
db.massShootings.insert_many(data3)

In [26]:
# Verify that the data can be pulled from document database (noSQL) Database is MonogoDB
massDbCol = db["massShootings"]   # Define variable to hold collection

x = massDbCol.find()   # define variable to hold list to loop through information to find data in collection

for data3 in x:   # use for loop to loop through records in database
    print(data3['State'], data3['# Killed'], data3['Incident Date'])  # pull the following information 

Pennsylvania 1 27-Sep-22
New York 0 27-Sep-22
Pennsylvania 0 25-Sep-22
Indiana 1 25-Sep-22
Ohio 1 24-Sep-22
Georgia 1 24-Sep-22
California 0 23-Sep-22
Illinois 0 22-Sep-22
New York 0 20-Sep-22
Michigan 2 20-Sep-22
Georgia 1 19-Sep-22
Wisconsin 2 18-Sep-22
Michigan 0 18-Sep-22
Nevada 0 18-Sep-22
New York 1 18-Sep-22
Missouri 0 17-Sep-22
Minnesota 0 17-Sep-22
Wisconsin 0 16-Sep-22
Wisconsin 0 14-Sep-22
Florida 0 14-Sep-22
California 0 14-Sep-22
Illinois 2 13-Sep-22
California 0 13-Sep-22
Oklahoma 0 12-Sep-22
New Mexico 0 11-Sep-22
Michigan 0 11-Sep-22
Michigan 0 11-Sep-22
North Carolina 1 11-Sep-22
Indiana 2 10-Sep-22
Kentucky 0 10-Sep-22
New York 0 9-Sep-22
Maryland 5 9-Sep-22
Minnesota 1 9-Sep-22
Minnesota 0 8-Sep-22
California 0 8-Sep-22
Tennessee 3 7-Sep-22
Missouri 2 5-Sep-22
Georgia 2 5-Sep-22
Ohio 1 5-Sep-22
Pennsylvania 2 5-Sep-22
Missouri 0 5-Sep-22
Minnesota 3 4-Sep-22
Virginia 2 4-Sep-22
Florida 0 4-Sep-22
Illinois 2 4-Sep-22
South Carolina 0 4-Sep-22
California 0 3-Sep-22
Ill

Tennessee 3 27-Jun-21
North Carolina 0 27-Jun-21
Texas 1 26-Jun-21
Illinois 1 26-Jun-21
Alabama 1 26-Jun-21
Ohio 0 25-Jun-21
Illinois 0 25-Jun-21
New York 0 25-Jun-21
Florida 1 24-Jun-21
Alabama 2 23-Jun-21
New York 0 23-Jun-21
Illinois 0 23-Jun-21
Illinois 0 23-Jun-21
Missouri 3 21-Jun-21
District of Columbia 0 21-Jun-21
Virginia 1 20-Jun-21
California 3 20-Jun-21
Colorado 1 20-Jun-21
Florida 1 20-Jun-21
Texas 0 20-Jun-21
Indiana 1 20-Jun-21
New Jersey 1 20-Jun-21
Ohio 1 20-Jun-21
Alaska 1 19-Jun-21
Louisiana 2 19-Jun-21
New Jersey 0 19-Jun-21
California 1 19-Jun-21
Minnesota 0 18-Jun-21
Virginia 0 17-Jun-21
North Carolina 2 17-Jun-21
Ohio 1 17-Jun-21
New Jersey 0 17-Jun-21
Maryland 1 16-Jun-21
Illinois 0 15-Jun-21
Illinois 5 15-Jun-21
Alabama 3 15-Jun-21
Georgia 2 14-Jun-21
Michigan 2 14-Jun-21
Florida 1 14-Jun-21
Ohio 1 13-Jun-21
Kentucky 1 13-Jun-21
Texas 1 12-Jun-21
Ohio 3 12-Jun-21
Illinois 1 12-Jun-21
Washington 2 11-Jun-21
Georgia 2 11-Jun-21
North Carolina 1 11-Jun-21
Texas 0 

North Carolina 0 20-Feb-20
Mississippi 2 17-Feb-20
Connecticut 1 16-Feb-20
Florida 1 16-Feb-20
Louisiana 0 15-Feb-20
Tennessee 0 15-Feb-20
Illinois 0 14-Feb-20
Delaware 1 10-Feb-20
Texas 1 8-Feb-20
Indiana 4 5-Feb-20
Maine 3 3-Feb-20
California 1 3-Feb-20
Mississippi 0 1-Feb-20
California 1 31-Jan-20
Louisiana 0 31-Jan-20
Florida 1 31-Jan-20
California 0 30-Jan-20
Connecticut 0 27-Jan-20
Missouri 0 26-Jan-20
New York 3 26-Jan-20
North Carolina 0 26-Jan-20
South Carolina 3 26-Jan-20
North Carolina 5 24-Jan-20
Washington 1 22-Jan-20
Missouri 2 19-Jan-20
Tennessee 1 19-Jan-20
Texas 2 19-Jan-20
Texas 2 18-Jan-20
Utah 4 17-Jan-20
Illinois 0 16-Jan-20
Colorado 0 11-Jan-20
Mississippi 2 8-Jan-20
Georgia 0 5-Jan-20
Florida 1 4-Jan-20
New York 0 2-Jan-20
Ohio 0 1-Jan-20
West Virginia 0 1-Jan-20
Missouri 3 1-Jan-20
Illinois 0 29-Dec-19
New York 1 29-Dec-19
California 0 29-Dec-19
California 1 27-Dec-19
Texas 2 27-Dec-19
Georgia 0 27-Dec-19
Florida 0 26-Dec-19
California 0 25-Dec-19
Iowa 1 25-Dec-